In [27]:
import cv2
import smtplib
from email.mime.text import MIMEText
import time
# import cv2.bgsegm
import psutil
import os

# Start time
start_time = time.time()

# Get process ID
process = psutil.Process(os.getpid())


cooldown_period = 300
last_trespass_alert_time = 0


# Initialize background subtractor
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)) 
def send_email_alert(
    sender = "your_email@example.com",
    receiver = "authority@example.com",
    subject = "Trespassing Alert",
    body = "A human trespassing event was detected at [Camera_ID] on [Timestamp]."
):
    global last_trespass_alert_time
    current_time = time.time()
    
    if current_time - last_trespass_alert_time >= cooldown_period:
        last_trespass_alert_time = current_time
        msg = MIMEText(body)
        msg['Subject'] = subject
        msg['From'] = sender
        msg['To'] = receiver
        try:
            server = smtplib.SMTP("smtp.example.com", 587)  # Change host and port as needed
            server.starttls()
            server.login(sender, "your_email_password")
            server.sendmail(sender, [receiver], msg.as_string())
            server.quit()
            print("Alert email sent.")
        except Exception as e:
            print("Failed to send email:", e)

# Sample loop for processing video frames
cap = cv2.VideoCapture(r"C:\Users\haridas\Desktop\PU\HT\smart-watch\v4.mp4")  # Replace with your video source

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fgMask = backSub.apply(frame)
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)
    # Additional processing: noise reduction and thresholding (details omitted for brevity)
    # Identify contours in the foreground mask
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if 1000 > cv2.contourArea(contour) > 800:  # Set area threshold to ignore small movements
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = h / float(w)
            # A simple check: humans tend to have an aspect ratio greater than 1.2
            if aspect_ratio > 1.2:
                # Optionally run a Haar-cascade human detector on this region if needed
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # print("Human-like motion detected.")
                # Trigger the email alert (ensure you throttle alerts to prevent spamming)
                # send_email_alert()
                # print("\n\nDETECTED\n\n")
                break  # Trigger one alert and continue processing

    # Display both fgMask and frame side by side
    cv2.imshow("FG Mask", fgMask)
    cv2.moveWindow("FG Mask", 0, 0)
    cv2.imshow("Frame", frame)
    cv2.moveWindow("Frame", 640, 0)
    # cv2.imshow("Frame", frame)
    # time.sleep(1)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
# End time
end_time = time.time()

# Get memory usage in MB
memory_used = process.memory_info().rss / (1024 * 1024)

print(f"Execution Time: {end_time - start_time:.2f} seconds")
print(f"Peak Memory Usage: {memory_used:.2f} MB")


Execution Time: 8.69 seconds
Peak Memory Usage: 146.74 MB


In [16]:
# Face Recognition

# Importing the libraries
import cv2

# Loading the cascades
face_cascade = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('Haarcascades/haarcascade_eye.xml')

# Defining a function that will do the detections
def detect(gray, frame):
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 6)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
    return frame

# Doing some Face Recognition with the webcam
video_capture = cv2.VideoCapture(r"C:\Users\haridas\Desktop\PU\HT\smart-watch\v7.mp4")  # Replace with your video source
# video_capture = cv2.VideoCapture()
while True:
    _, frame = video_capture.read()
    if not _: break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    canvas = detect(gray, frame)
    cv2.imshow('Video', canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()


In [7]:
import cv2
import smtplib
from email.mime.text import MIMEText
import time
import psutil
import os

# Start time
start_time = time.time()

# Get process ID
process = psutil.Process(os.getpid())

cooldown_period = 300
last_trespass_alert_time = 0

# Initialize background subtractor
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)) 

# Video source
input_video_path = r"C:\Users\haridas\Desktop\PU\HT\smart-watch\v1.mp4"  # Replace with your video path
output_video_path = r"C:\Users\haridas\Desktop\PU\HT\smart-watch\output.mp4"  # Path to save processed video

cap = cv2.VideoCapture(input_video_path)

# Get video properties (width, height, fps)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' for AVI
# out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
output_width = frame_width * 2  # Double the width for side-by-side display
output_height = frame_height    # Height remains the same

out = cv2.VideoWriter(output_video_path, fourcc, fps, (output_width, output_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fgMask = backSub.apply(frame)
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)
    fgMask[fgMask == 127] = 0  # Remove shadows
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # Remove noise

    # Identify contours
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if 1000 > cv2.contourArea(contour) > 800:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = h / float(w)
            if aspect_ratio > 1.2:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # print("Human-like motion detected.")
                break  

    # Write the processed frame to the output video
    fgMask_colored = cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)  # Convert grayscale fgMask to 3-channel
    combined_frame = cv2.hconcat([fgMask_colored, frame])  # Stack horizontally

    out.write(combined_frame)

    # Display frames
    cv2.imshow("FG Mask", fgMask)
    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) == ord('q'):
        break

# Release resources
cap.release()
out.release()  # Save the video
cv2.destroyAllWindows()

# End time
end_time = time.time()

# Get memory usage in MB
memory_used = process.memory_info().rss / (1024 * 1024)

print(f"Execution Time: {end_time - start_time:.2f} seconds")
print(f"Peak Memory Usage: {memory_used:.2f} MB")
print(f"Processed video saved at: {output_video_path}")


Execution Time: 11.98 seconds
Peak Memory Usage: 63.00 MB
Processed video saved at: C:\Users\haridas\Desktop\PU\HT\smart-watch\output.mp4


In [4]:
import cv2
import numpy as np

# Initialize video capture
cap = cv2.VideoCapture(r"C:\Users\haridas\Desktop\PU\HT\smart-watch\v5.mp4")  # Replace with your video

# Initialize Background Subtractor
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

# Initialize HOG descriptor + SVM for human detection
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Morphological kernel (to reduce noise)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply background subtraction
    fgMask = backSub.apply(gray)
    fgMask[fgMask == 127] = 0  # Remove shadows
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # Reduce noise

    # Find contours of moving objects
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    human_detected = False

    for contour in contours:
        if cv2.contourArea(contour) < 800:  # Ignore small movements
            continue

        # Get bounding box of moving object
        x, y, w, h = cv2.boundingRect(contour)
        # Ensure the extracted moving region is valid
        if w > 64 and h > 128:  # HOG requires at least 64x128 pixels to detect humans
            moving_region = frame[y:y+h, x:x+w]  # Extract moving object
            humans, _ = hog.detectMultiScale(moving_region, winStride=(4, 4), padding=(8, 8), scale=1.05)

            for (hx, hy, hw, hh) in humans:
                human_detected = True
                cv2.rectangle(frame, (x+hx, y+hy), (x+hx+hw, y+hy+hh), (0, 255, 0), 2)  # Draw human box
    # Print message if human motion is detected
    if human_detected:
        print("🚨 Human motion detected!")

    # Display results
    cv2.imshow("Foreground Mask", fgMask)
    cv2.imshow("Motion Detection", frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [9]:
from ultralytics import YOLO
import cv2

# Load YOLOv8 model (smallest version)
model = YOLO("yolov8n.pt")  # "n" means Nano, fastest model

# Open video stream
cap = cv2.VideoCapture(r"C:\Users\haridas\Desktop\PU\HT\smart-watch\v9.mp4")  # Use 0 for webcam or video file path

i = 1
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if i%5 != 0:
        i+=1
        continue
    i=1
    # Run YOLOv8 detection
    results = model(frame)

    human_detected = False
    for result in results:
        for box in result.boxes:
            if box.cls == 0:  # Class 0 in YOLO is "person"
                human_detected = True
                # print(box)
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Print result
    # print("🚨 Human detected!" if human_detected else "❌ No human detected.")

    # Show frame
    cv2.imshow("YOLOv8 Human Detection", frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 1 boat, 2 dogs, 188.2ms
Speed: 4.0ms preprocess, 188.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 dog, 1 handbag, 1 cup, 204.5ms
Speed: 5.5ms preprocess, 204.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 dog, 1 handbag, 1 cup, 177.5ms
Speed: 3.3ms preprocess, 177.5ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 dog, 1 handbag, 165.7ms
Speed: 3.0ms preprocess, 165.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 dog, 150.8ms
Speed: 3.0ms preprocess, 150.8ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 dog, 137.7ms
Speed: 1.0ms preprocess, 137.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 dog, 1 handbag, 165.1ms
Speed: 2.5ms preprocess, 165.1ms inference, 2.4ms postprocess per image at shape (1, 3